In [1]:
import easyocr, cv2, os
import numpy as np

In [2]:
class Scan:
    def __init__(self, oriImg) :
        self.oriImg = oriImg


    def adjust(self, img):
        # # 이미지 전처리 및 외곽선 추출
        edged = self.extractEdge(img)
        
        contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        draw = img.copy()
        cv2.drawContours(draw,  contours, -1, (0, 255, 0))
        
        # 사각형 중 최대크기의 컨투어 꼭지점
        pts = self.getPointsOfMaxRectangle(contours)

        # 각각의 좌표 찾기
        sumXY = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)

        topLeft = pts[np.argmin(sumXY)]
        bottomRight = pts[np.argmax(sumXY)]
        topRight = pts[np.argmin(diff)]
        bottomLeft = pts[np.argmax(diff)]

        # 사진을 변환할 때 사용할 서류의 높이
        widthTop = abs(topRight[0] - topLeft[0])
        widthBottom = abs(bottomRight[0] - bottomLeft[0])
        heightRight = abs(topRight[1] - bottomRight[1])
        heightLeft = abs(topLeft[1] - bottomLeft[1])
        print(widthBottom, widthTop, heightLeft, heightRight)

        width = max([widthTop, widthBottom])
        height = max([heightRight, heightLeft])

        pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
        pts2 = np.float32([[0,0], [width, 0], [width, height], [0, height]])

        matrix = cv2.getPerspectiveTransform(pts1, pts2) # 좌표를 변환하기 위해 사용할 변환행렬
        result = cv2.warpPerspective(img, matrix, (width, height)) # 이미지 변환(변환행렬 적용)

        return result
    
    def extractEdge(self, img):
        # 이미지 전처리 및 외곽선 추출
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0) # 이미지를 흐리게 처리함 (noise 제거를 위해 사용)
        edged = cv2.Canny(gray, 75, 250) # edged를 검출하는 함수 (img, minVal, maxVal)
        return edged
    
    def getPointsOfMaxRectangle(self, contours) :
        # 크기순으로 컨투어 정렬
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
        for c in contours:
            peri = cv2.arcLength(c, True) # 외곽선 길이
            # print(peri)
            verticles = cv2.approxPolyDP(c, 0.02 * peri, closed=True) # 외곽선 근사화
            if len(verticles) == 4 : 
                break
        pts = verticles.reshape(4, 2) # 배열을 4 * 2 크기로 조정
        return pts





In [3]:
img = cv2.imread("./../dataset/KakaoTalk_20240420_113433452_01.jpg")
scanner = Scan(img)

result = scanner.adjust(img)

cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

2317 1969 2939 3018


In [65]:
class Ocr:
    def getTextBoxes(self, img) : 
        contourList = []
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        # Morphology Transform - 1차
        kernel  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        grad = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)
        # Morphology Transform - 2차
        _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (20,3))
        connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel2)
        cv2.namedWindow("connected", cv2.WINDOW_NORMAL)
        cv2.imshow("connected", connected)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        # contour 찾기
        contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        mask = np.zeros(bw.shape, dtype=np.uint8)
        rgb = img.copy()
        for idx in range(len(contours)):
            x, y, w, h = cv2.boundingRect(contours[idx])
            mask[y:y+h, x:x+w] = 0
            cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
            r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w*h)
            # print(str(cv2.countNonZero(mask[y:y+h, x:x+w])) + " / " + str(w*h))
            if r > 0.01 and w > 8 and h > 8 :
                cv2.rectangle(rgb, (x,y), (x+w, y+h), (0, 255, 0), 2)
                contourList.append(contours[idx])
        # cv2.imshow("result", rgb)
        return (rgb, contourList)

In [66]:
ocr = Ocr()
_, list = ocr.getTextBoxes(result)
# print(list)



In [67]:
cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.imshow("img", _)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [153]:
# y축 기준으로 정렬하기
sortedContours = sorted(list, key=lambda y: cv2.boundingRect(y)[1], reverse=False)
# print(a)
# print(_.shape)
tmp = []
draw = _.copy()
for i in range(10):
    x, y, w, h = cv2.boundingRect(sortedContours[i])
    print(y, y+h)
    # if r > 0.01 and w > 8 and h > 8 :
    cv2.rectangle(draw, (x,y), (x+w, y+h), (255, 255, 0), 2)
# cv2.drawContours(draw,  contours, -1, (255, 0, 255), 2)
cv2.namedWindow("ddd", cv2.WINDOW_NORMAL)
cv2.imshow("ddd", draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

76 112
91 1837
154 205
163 208
194 247
203 248
237 285
244 289
273 325
285 329


In [154]:
# boundingRect가 위아래로 겹치는지 확인하기

# boundingRect의 평균높이 구하기
arrHeight = []
for el in sortedContours:
    arrHeight.append(cv2.boundingRect(el)[3])
meanHeight = np.mean(arrHeight) + 10


# contour 반으로 나눠보자
leftList = []
rightList = []
halfOfWidth = _.shape[1] / 2
for el in sortedContours:
    if (cv2.boundingRect(el)[0] < halfOfWidth) :
        leftList.append(el)
    else:
        rightList.append(el)

draw = _.copy()
startContours = []
endContours = []
prev = cv2.boundingRect(rightList[1])   # 일단 예외처리 (2번째까지는 생략)
startContours.append(prev)
for i in range(len(rightList)):
    if (i <= 0) :    # 일단 예외처리 (2번째까지는 생략)
        continue
    x, y, w, h = cv2.boundingRect(rightList[i])
    prevHeight = prev[1] + prev[3]
    margin = prevHeight + meanHeight
    if (h > meanHeight * 3) :
        cv2.rectangle(draw, (x, y), (x+w, y+h), (111, 111, 111), 10)
        continue
    if(y+h > margin):
        # print(prev, margin)
        endContours.append(prev)
        startContours.append(prev)
    prev = (x, y, w, h)
# print(startContours)
# print(endContours)
# print(len(a))
# cv2.boundingRect(a[0])
# _.shape


# draw = _.copy()
for i in range(len(startContours)-1):
    startX = min(startContours[i][0], endContours[i][0])
    startY = min(startContours[i][1], endContours[i][1])
    endX = max(startContours[i][0]+startContours[i][2], endContours[i][0]+endContours[i][2])
    endY = max(startContours[i][1]+startContours[i][3], endContours[i][1]+endContours[i][3])
    cv2.rectangle(draw, (startX, startY), (endX, endY), (255, 255, 255), 3)

# cv2.rectangle(draw, (2308, 91), (2308+9, 91+1746), (255, 255, 0), 2)
cv2.namedWindow("ddd", cv2.WINDOW_NORMAL)
cv2.imshow("ddd", draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
print(_.shape)
tmp = []
for i in range(len(list)):
    x, y, w, h = cv2.boundingRect(list[i])
    print(y, y+h)

print(3018/2317)

(3018, 2317, 3)
3006 3018
2700 2733
2618 2653
2586 2662
2579 2614
2545 2614
2501 2534
2429 2544
2426 2457
2346 2422
1.3025463962019854


In [63]:

# # 이미지 전처리 및 외곽선 추출
gray = cv2.cvtColor(_, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 이미지를 흐리게 처리함 (noise 제거를 위해 사용)
edged = cv2.Canny(gray, 75, 250) # edged를 검출하는 함수 (img, minVal, maxVal)

contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
draw = _.copy()
cv2.drawContours(draw,  contours, -1, (255, 0, 255), 2)
cv2.imshow("ddd", draw)
cv2.waitKey(0)
cv2.destroyAllWindows()